In [2]:
from math import exp
from random import seed
from random import random
def initialize_network(n_input,n_hidden,n_output):
    network=list()
    hidden_layer=[{'weights':[random() for i in range (n_inputs+1)]}for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer=[{'weights':[random() for i in range (n_hidden+1)]}for i in range(n_outputs)]
    network.append(output_layer)
    return network

def activate(weights,inputs):
    activation=weights[-1]
    for i in range(len(weights)-1):
        activation+=weights[i]*inputs[i]
    return activation

def transfer(activation):
    return 1.0/(1.0+exp(-activation))

def forword_propogate(network,row):
    inputs=row
    for layer in network:
        new_inputs=[]
        for neuron in layer:
            activation=activate(neuron['weights'],inputs)
            neuron['output']=transfer(activation)
            new_inputs.append(neuron['output'])
        inputs=new_inputs
    return inputs


def transfer_derivative(output):
    return output*(1.0-output)

def backward_propogate_error(network,expected):
    for i in reversed(range(len(network))):
        layer=network[i]
        errors=list()
        if i!=len(network)-1:
            for j in range(len(layer)):
                error=0.0
                for neuron in network[i+1]:
                    error+=(neuron['weights'][j]*neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron=layer[j]
                errors.append(expected[j]-neuron['output'])
        for j in range(len(layer)):
            neuron=layer[j]
            neuron['delta']=errors[j]*transfer_derivative(neuron['output'])
            
def update_weights(network,row,l_rate):
    for i in range(len(network)):
        inputs=row[:-1]
        if i!=0:
            inputs=[neuron['output']for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j]+=l_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1]+=l_rate*neuron['delta']
            
def train_network(network,train,l_rate,n_epoch,n_outputs):
    for epoch in range(n_epoch):
        sum_error=0
        for row in train:
            outputs=forword_propogate(network,row)
            expected=[0 for i in range(n_outputs)]
            expected[row[-1]]=1
            sum_error+=sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propogate_error(network,expected)
            update_weights(network,row,l_rate)
        print('>epoch=%d,lrate=%.3f,error=%.3f' %(epoch,l_rate,sum_error))
        
seed(1)
dataset=[[2.7810836,2.550537003,0],
        [1.465480372,2.362125076,0],
        [3.396561688,4.400293529,0],
        [1.38807019,1.850220317,0],
        [3.06407232,3.005305973,0],
        [7.627531214,2.759262235,1],
        [5.332441248,2.0886267745,1],
        [6.922596716,1.77106367,1],
        [8.675418651,-0.242068655,1],
        [7.673756466,3.508563011,1]]
n_inputs=len(dataset[0])-1
n_outputs=len(set([row[-1]for row in dataset]))
network=initialize_network(n_inputs,2,n_outputs)
print(network)
train_network(network,dataset,0.5,5,n_outputs)
print('the layers are as follows')
for layer in network:
    print(layer)

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]}], [{'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349]}, {'weights': [0.02834747652200631, 0.8357651039198697, 0.43276706790505337]}]]
>epoch=0,lrate=0.500,error=6.350
>epoch=1,lrate=0.500,error=5.531
>epoch=2,lrate=0.500,error=5.221
>epoch=3,lrate=0.500,error=4.951
>epoch=4,lrate=0.500,error=4.519
the layers are as follows
[{'weights': [-0.7584213742050507, 0.72898953144139, 0.6869088970041115], 'output': 0.09768733634840576, 'delta': -0.00975495775963815}, {'weights': [0.0587167558732027, 0.3232604704946165, 0.3820731345234498], 'output': 0.8692761713542836, 'delta': 0.0020016742556771476}]
[{'weights': [0.46642753086252525, 0.22320934644218837, -0.5576944760190755], 'output': 0.44494568385158106, 'delta': -0.10988780050560763}, {'weights': [-0.6383740805521387, 0.5576637435280111, 0.12219773028557465], 'output': 0